<a href="https://colab.research.google.com/github/objectis/ai-pneumonia-detection/blob/master/detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing images

In [0]:
import os
import cv2
import pickle
import random
import numpy as np
import matplotlib.pyplot as plt

In [10]:
os.environ['KAGGLE_USERNAME'] = "xxx" 
os.environ['KAGGLE_KEY'] = "xxx" 
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 1.15G/1.15G [00:09<00:00, 150MB/s]
100% 1.15G/1.15G [00:09<00:00, 132MB/s]


Extracting downloaded images

In [0]:
from zipfile import ZipFile

zfile = ZipFile('chest_xray.zip', 'r')
zfile.extractall()
zfile.close()

defining file paths

In [52]:
from pathlib import Path

train_files = 'chest_xray/train'
test_files = 'chest_xray/test'
val_files = 'chest_xray/val'

CLASSES = ['PNEUMONIA','NORMAL']
def countFiles(path,categories):
  for cat in categories:
    folder = Path(path + "/" + cat + "/")
    tmp = list(folder.glob("*.jpeg"))
    print(path, "     ", cat, " : " ,len(tmp))
  return

countFiles(train_files,CLASSES)
countFiles(test_files,CLASSES)
countFiles(val_files,CLASSES)


chest_xray/train       PNEUMONIA  :  3875
chest_xray/train       NORMAL  :  1341
chest_xray/test       PNEUMONIA  :  390
chest_xray/test       NORMAL  :  234
chest_xray/val       PNEUMONIA  :  8
chest_xray/val       NORMAL  :  8


pre-processing data for the model using ImageDataGenerator

In [0]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.enable_eager_execution()
AUTO = tf.data.experimental.AUTOTUNE

img_gen = ImageDataGenerator(featurewise_center = True,
                              shear_range = 0.2,
                              rescale = 1.0/255,
                              horizontal_flip = True)



In [62]:
to_train = img_gen.flow_from_directory(train_files, 
                                        target_size = (150, 150),
                                        classes = CLASSES,
                                        batch_size = 256, 
                                        shuffle = True, 
                                        class_mode = "binary")
to_test = img_gen.flow_from_directory(test_files, 
                                        target_size = (150, 150),
                                        classes = CLASSES,
                                        batch_size = 256, 
                                        shuffle = True, 
                                        class_mode = "binary")
to_val = img_gen.flow_from_directory(val_files, 
                                        target_size = (150, 150),
                                        classes = CLASSES,
                                        batch_size = 256, 
                                        shuffle = True, 
                                        class_mode = "binary")


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [0]:
from sklearn.utils.class_weight import compute_class_weight

y = to_train.classes
labels = np.unique(y)

train_class_weights = compute_class_weight("balanced", labels, y)

#Machine learning

In [0]:
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, Dropout, GlobalAveragePooling1D, GlobalAveragePooling2D, Flatten, BatchNormalization, Dense

from tensorflow.keras.models import Sequential, load_model, save_model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop


In [73]:
model = InceptionV3(weights = "imagenet", include_top = False, input_shape = (150, 150, 3))
model.trainable = False 

xray_model = Sequential([model,
                      GlobalAveragePooling2D(),
                      Dense(1024, activation = "relu"),
                      Dropout(0.3),
                      Dense(512, activation = "relu"),
                      Dropout(0.3),
                      Dense(256, activation = "relu"),
                      Dropout(0.3),
                      Dense(128, activation = "relu"),
                      Dropout(0.3),
                      Dense(1, activation = "sigmoid")])

xray_model.compile(RMSprop(), loss = "binary_crossentropy", metrics = ["accuracy"])
xray_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 3, 3, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dropout_8 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 256)              